In [1]:
# world, players, location, type, activity

# 1 - extract 'world', 'location', 'type', 'activity'
# 2 - extract 'world', 'datetime', 'players'

'\n<tr class="server-list__row">\n<td class="server-list__row-cell"> # world below\n<a class="server-list__world-link" href="https://oldschool.runescape.com/game?world=468" id="slu-world-468">OldSchool 168</a>\n</td>\n<td class="server-list__row-cell">85 players</td> # players\n<td class="server-list__row-cell server-list__row-cell--country server-list__row-cell--US">United States</td> # location\n<td class="server-list__row-cell server-list__row-cell--type">Free</td> # type\n<td class="server-list__row-cell">500 skill total</td> # activity\n</tr>\n'

In [27]:
import numpy as np
import pandas as pd
import re

from bs4 import BeautifulSoup

In [146]:
def drop_even_columns(data):
    data = data[data.columns[1::2]]
    return data


def select(data, columns):
    data = data[columns]
    return data


def rename_columns(data):
    columns = ['world', 'players', 'location', 'type', 'activity']
    new_columns = dict(zip(data.columns, columns))
    renamed_data = data.rename(columns=new_columns)
    return renamed_data


def get_content(column, index=0):
    content = column.apply(lambda tag: tag.contents[index])
    return content


def get_contents(data):
    data['world'] = get_content(data['world'], index=1)
    for column in data.columns:
        data[column] = get_content(data[column])
        
    return data


def get_number(string):
    pattern = re.compile(r'\d+')
    match = re.search(pattern, string)
    number = match.group(0)
    return number


def get_numbers(data):
    columns = ['world', 'players']
    for column in columns:
        data[column] = data[column].apply(get_number)
        data[column] = data[column].astype(np.int16)

    return data


def transform_data(data, columns):
    data = data.copy()
    transformed_data = (
        data.pipe(drop_even_columns)
            .pipe(rename_columns)
            .pipe(get_contents)
            .pipe(get_numbers)
            .pipe(select, columns=columns)
    )
    return transformed_data

In [4]:
with open('osrs_worlds.html', 'rb') as page:
    page_content = page.read()
    soup = BeautifulSoup(page_content, 'lxml')

In [143]:
row_data = soup.findAll('tr', {'class':'server-list__row'})
zeta = pd.DataFrame(np.array(row_data, dtype=object))

In [147]:
transformed_data = transform_data(zeta, columns=['world', 'players'])
transformed_data.head(3)

,world,players
0,168,85
1,117,179
2,183,181


In [158]:
world_info = transform_data(zeta, columns=['world', 'location', 'type', 'activity'])
world_info

,world,location,type,activity
0,168,United States,Free,500 skill total
1,117,United States,Free,-
2,183,United States,Free,-
3,94,United States,Free,Clan Wars - Free
4,170,United States,Free,-
...,...,...,...,...
258,88,Australia,Members,Theatre of Blood
259,124,Australia,Members,Blast Furnace
260,87,Australia,Members,Blast Furnace
261,89,Australia,Members,Wintertodt
